In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
"""
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cv2                 
import numpy as np         
import os                  
from random import shuffle
import tensorflow as tf 
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob as gb
from tensorflow.keras.utils import to_categorical

In [ ]:
#read DataSet
train_image="/kaggle/input/chest-xray-covid19-pneumonia/Data/train/"
test_image="/kaggle/input/chest-xray-covid19-pneumonia/Data/test/"

In [ ]:
print(train_image)
print(test_image)

In [ ]:
#to get all image names in train file
pneumona_images = os.listdir(train_image + "/PNEUMONIA")
normal_images = os.listdir(train_image + "/NORMAL")
covid_images = os.listdir(train_image + "/COVID19")

In [ ]:
#plot to show the size of some image
#plot PNEUMONIA
plt.figure(figsize=(20,10))
for i in range(6):
    plt.subplot(3, 3, i + 1)
    plt.imshow(plt.imread(os.path.join(train_image + "/PNEUMONIA",pneumona_images[i])),cmap='gray')
    plt.title("PNEUMONIA")
    
plt.show()
#plot NORMAL
plt.figure(figsize=(20,10))
for i in range(6):
    plt.subplot(3, 3, i + 1)
    plt.imshow(plt.imread(os.path.join(train_image + "/NORMAL",normal_images[i])),cmap='gray')
    plt.title("NORMAL")

plt.show()
#plot 
plt.figure(figsize=(20,10))
for i in range(6):
    plt.subplot(3, 3, i + 1)
    plt.imshow(plt.imread(os.path.join(train_image + "/COVID19",covid_images[i])),cmap='gray')
    plt.title("COVID19")

In [ ]:
train_datagen = ImageDataGenerator(
      horizontal_flip=True,
      rotation_range=10,
      zoom_range=0.4)

train_generator =train_datagen.flow_from_directory(
     train_image,
     batch_size= 256,
     shuffle=shuffle,
     target_size=(256, 256))

test_generator =train_datagen.flow_from_directory(
     test_image,
     batch_size= 50,
     shuffle=shuffle,
     target_size=(256, 256)) 

In [ ]:
train_shape=train_generator.__getitem__(0)[0].shape
test_shape=test_generator.__getitem__(0)[0].shape
#Shape of Data
print("Train Shape \n",train_shape)
print("Test Shape \n",test_shape)

In [ ]:
Labels={'NORMAL':0,'PNEUMONIA':1,'COVID19':2}

# convert label to code
def getCode(label):
    return Labels[label]


# convert code to label 
def getLabel(n):
    for x,c in Labels.items():
        if n==c:
            return x   
        
#Test        
print(getCode('COVID19'))
print(getLabel(1))

In [ ]:
#Reading image data
import glob as gb
import cv2  
size_image=256 # to resize the all image as same size

#to read all images from directory
def getData(Dir,size_image):
    X=[]
    y=[]
    for folder in  os.listdir(Dir) : #to get the file name 
        files = gb.glob(pathname= str( Dir  +"/" +folder+ '//*.jpg' )) # to get the images
        for file in files:
                picture=cv2.imread(file) #  or plt.imread(file)
                imageArray=cv2.resize(picture,(size_image,size_image))
                X.append(list(imageArray))
                y.append(getCode(folder))
    X=np.array(X)
    y=np.array(y)
    return X,y

In [ ]:
#get train data
X_train, y_train = getData(train_image,size_image)
# get test data
X_test , y_test = getData(test_image,size_image)

In [ ]:
print("X_train Shape        ",X_train.shape)
print("X_test Shape         ",X_test.shape)

In [ ]:
# #Convert y_train to categorical
y_train=to_categorical(y_train,3)
print("y_train ",y_train.shape)

#Convert y_train to categorical
y_test=to_categorical(y_test,3)
print("y_test ",y_test.shape)

# Generalized Model 

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import LeakyReLU #, Dense, Dropout, Flatten
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [ ]:
def disease_prediction_model(pre_trained_model,file_name):
    for layer in pre_trained_model.layers:
        layer.trainable = False  #to make the layers to Freeze Weights
    pre_trained_model.summary()
    
    
    """
    model = Sequential()
    model.add(Flatten(pre_trained_model.output))
    #Full Connected Layers
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.1))
    #Add dropout to avoid Overfit
    model.add(Dropout(0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.1))
    
    #Add dropout to avoid Overfit
    model.add(Dropout(0.4))
    model.add(Dense(64))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(3, activation='softmax'))
    #x=tf.keras.layers.Dense(3 , activation='softmax')(x) # Softmax

    """
    
    x = tf.keras.layers.Flatten()(pre_trained_model.output)

    #Full Connected Layers
    x = tf.keras.layers.Dense(512, activation=LeakyReLU(alpha=0.01))(x)
    #Add dropout to avoid Overfit
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(256, activation=LeakyReLU(alpha=0.01))(x) # Leaky ReLu
    x = tf.keras.layers.Dense(128, activation=LeakyReLU(alpha=0.01))(x) # Leaky ReLu
    #Add dropout to avoid Overfit
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(64, activation=LeakyReLU(alpha=0.01))(x) # Leaky ReLu


    x=tf.keras.layers.Dense(3 , activation='softmax')(x) # Softmax
    
    model = Model( pre_trained_model.input, x) 

    print(model.summary())
    model.compile(optimizer='adam', loss="categorical_crossentropy",metrics=['accuracy'])
    
    epochs = 30
    history = model.fit_generator(train_generator, steps_per_epoch=2, epochs=epochs)
    
    plt.plot(history.history['accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='upper left')
    plt.show()
    
    #Evaluate Model
    model.evaluate(test_generator)

    model.save(file_name+".h5")

    #prediction
    pred=model.predict(test_generator)
    print(len(pred))

    y_test=[]
    for i in range(26):
        y_test.extend(test_generator.__getitem__(i)[1])
        
    print(len(y_test))
    y_test=np.array(y_test)

    y_test=np.argmax(y_test,axis=1)
    pred= np.argmax(pred,axis=1)

    print("pred \n",len(pred))
    print("y_test \n",len(y_test))
    
    print("y_test \n",y_test)
    print("predicted \n",pred)
    
    cm=confusion_matrix(pred,y_test)
    print(cm)
    
    """
    plt.figure(figsize=(20,10))
    for i in range(0,9):

        plt.subplot(3, 3, i + 1)

        plt.imshow(test_generator.__getitem__(0)[0][i],cmap='gray')
        plt.title(f"   Real: {getLabel(y_test[i])   } Vs  Predict: {getLabel(pred[i])}")

    """
    
    precision = precision_score(y_test,pred, average="weighted")
    print('Precision: %f' % precision)
    recall = recall_score(y_test,pred,average="weighted")
    print('Recall: %f' % recall)
    f1 = f1_score(y_test,pred,average="weighted")
    print('F1 score: %f' % f1)
    
    pass

In [ ]:
from tensorflow.keras.applications import InceptionV3 as InceptionV3
from tensorflow.keras.applications import Xception as Xception
from tensorflow.keras.applications import ResNet50 as ResNet

In [ ]:
pre_trained_model = InceptionV3(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=(size_image,size_image,3), pooling=None, classes=3,
    classifier_activation='softmax'
)

disease_prediction_model(pre_trained_model,'Covid19-inception')

In [ ]:
pre_trained_model = Xception(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=(size_image,size_image,3), pooling=None, classes=3,
    classifier_activation='softmax'
)

disease_prediction_model(pre_trained_model,'covid19-xception')

In [ ]:
pre_trained_model = ResNet(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=(size_image,size_image,3), pooling=None, classes=3,
    classifier_activation='softmax'
)

disease_prediction_model(pre_trained_model,'covid19-resnet')